#### import libraries

In [1]:
# import libraries
from __future__ import print_function
import os
import numpy as np
import _pickle as pickle
from tensorflow import keras
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten

#### load dataset

In [2]:
data_list = []
label_list = []
for i in range(1,2):
    # select filepath
    filepath  = "C:\\Users\Kuldeep Jaimini\Desktop\project_work\cifar_code\cifar-10-batches-py\data_batch_"+str(i)
    # open filepath
    file = open(filepath, "rb")
    #load data
    data_dict = pickle.load(file, encoding = "latin1")
    data = data_dict["data"]
    label = data_dict["labels"]
    data_list.append(data)
    label_list.append(label)
# total data    
data = np.concatenate(data_list)
# correesponding labels
label = np.concatenate(label_list)

#### shuffle the dataset

In [3]:
indices = np.arange(data.shape[0], dtype=np.int32)
np.random.shuffle(indices)
data = data[indices]
label = label[indices]

#### shape of the dataset

In [4]:
print("shape of the data:",data.shape)
print("shape of the label:",label.shape)

shape of the data: (10000, 3072)
shape of the label: (10000,)


#### normalize the dataset

In [5]:
data = data/255.0

#### split the dataset into train and test set

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.20, random_state=42)
print("shape of training data",x_train.shape)
print("shape of test data", x_test.shape)

shape of training data (8000, 3072)
shape of test data (2000, 3072)


In [7]:
batch_size = 32
num_classes = 10
epochs = 50
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [8]:
# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

(8000, 10)
(2000, 10)


#### create model

In [9]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same',
                 input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# initiate RMS prop optimizer
import tensorflow
from tensorflow.keras import optimizers
opt = tensorflow.keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
# Let's train the model using RMS prop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [11]:
x_train = x_train.reshape(-1,32,32,3)
x_test = x_test.reshape(-1,32,32,3)
print(x_train.shape)

(8000, 32, 32, 3)


In [12]:
model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=False)

Train on 8000 samples, validate on 2000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
8000/8000 [==============================] - 26s 3ms/sample - loss: 2.0567 - acc: 0.2461 - val_loss: 1.8010 - val_acc: 0.3500
Epoch 2/50
8000/8000 [==============================] - 27s 3ms/sample - loss: 1.7483 - acc: 0.3704 - val_loss: 1.5944 - val_acc: 0.4105
Epoch 3/50
8000/8000 [==============================] - 27s 3ms/sample - loss: 1.5922 - acc: 0.4238 - val_loss: 1.4729 - val_acc: 0.4545
Epoch 4/50
8000/8000 [==============================] - 28s 3ms/sample - loss: 1.4934 - acc: 0.4656 - val_loss: 1.4340 - val_acc: 0.4750
Epoch 5/50
8000/8000 [==============================] - 27s 3ms/sample - loss: 1.4063 - acc: 0.4961 - val_loss: 1.3687 - val_acc: 0.4940
Epoch 6/50
8000/8000 [==============================] - 27s 3ms/sample - loss: 1.3268 - acc: 0.5288 - val_loss: 1.3375 - val_acc: 0.5170
Epoch 7/50
8000/8000 [==============================] - 28s 3ms/sample - loss: 1.2

In [13]:
# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

2000/2000 [==============================] - 3s 2ms/sample - loss: 1.9176 - acc: 0.5170
Test loss: 1.9176317539215089
Test accuracy: 0.517
